# **Machine Learning Techniques Project**


*   Nicolas Bedoya Figueroa
*   Daniel Escalante Perez
*   Marilyn Stephany Joven Fonseca
*   Eder Leandro Carbonero Baquero

## **Utils**

In [1]:
!pip install nltk pyspellchecker tqdm emoji nlpaug transformers tensorflow xgboost[gpu] accelerate

In [2]:
!pip install seaborn transformers torch tqdm 

In [3]:
!pip install umap-learn plotly

In [4]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from spellchecker import SpellChecker
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tqdm import tqdm
import nlpaug.augmenter.word as naw
from nlpaug.util import Action
import emoji
import random
import math
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')

2025-05-16 00:53:46.071624: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-16 00:53:46.078848: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747374826.087083    3072 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747374826.089607    3072 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747374826.096162    3072 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

True

Librerias ML y DL

In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModel
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import umap
import plotly.express as px
import ast
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import warnings
import random
from sklearn.metrics.pairwise import cosine_similarity


In [6]:
random_seed = 611
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

## **Data preprocessing**

### **Dataset 1: Davidson et al. 2017**

In [6]:
# Load the dataset
davidson = pd.read_csv("./data/davidson_2017.csv")[["class", "tweet"]]
davidson.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [7]:
# Original class labels: 0 - hate speech, 1 - offensive language, 2 - neither
# Transform the label in 1: toxic and 0: non toxic
davidson["class"] = davidson["class"].replace({0: 1, 2: 0})
davidson["class"].value_counts()

class
1    20620
0     4163
Name: count, dtype: int64

In [8]:
print("Rows:", davidson.shape[0])
print("Columns:", davidson.shape[1])

Rows: 24783
Columns: 2


### **Dataset 2: HASOC (2019) English**

In [9]:
hasoc = pd.read_csv("./data/HASOC_EN.tsv",sep = '\t')[["text","task_1"]]
hasoc.head()

,text,task_1
0,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT
1,@politico No. We should remember very clearly ...,HOF
2,@cricketworldcup Guess who would be the winner...,NOT
3,Corbyn is too politically intellectual for #Bo...,NOT
4,All the best to #TeamIndia for another swimmin...,NOT


In [10]:
# Transform the label to 1: toxic and 0: non toxic
hasoc["task_1"] = hasoc["task_1"].replace({"HOF": 1, "NOT": 0})
hasoc["task_1"].value_counts()

/tmp/ipykernel_20878/1216868155.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  hasoc["task_1"] = hasoc["task_1"].replace({"HOF": 1, "NOT": 0})


task_1
0    3591
1    2261
Name: count, dtype: int64

In [11]:
# Change column names to match the other datasets
hasoc = hasoc.rename(columns={'task_1': 'class', 'text': 'tweet'})
hasoc.head()

,tweet,class
0,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,0
1,@politico No. We should remember very clearly ...,1
2,@cricketworldcup Guess who would be the winner...,0
3,Corbyn is too politically intellectual for #Bo...,0
4,All the best to #TeamIndia for another swimmin...,0


In [12]:
print("Rows:", hasoc.shape[0])
print("Columns:", hasoc.shape[1])

Rows: 5852
Columns: 2


### **Dataset 3: Zeerak Talat’s Hate Speech Dataset**

In [13]:
# Load the dataset
zeerak = pd.read_csv("./data/NAACL_SRW_2016_fixed.csv")[["class", "text"]]
zeerak.head()

,class,text
0,racism,So Drasko just said he was impressed the girls...
1,racism,Drasko they didn't cook half a bird you idiot ...
2,racism,Hopefully someone cooks Drasko in the next ep ...
3,racism,of course you were born in serbia...you're as ...
4,racism,RT @YesYoureRacist: At least you're only a tin...


In [14]:
# Original class labels
zeerak["class"].value_counts()

class
none      7060
sexism    2577
racism      11
Name: count, dtype: int64

In [15]:
# Transform the label to 1: toxic and 0: non toxic
zeerak["class"] = zeerak["class"].replace({"sexism": 1, "racism": 1, "none": 0})
zeerak["class"].value_counts()

/tmp/ipykernel_20878/990906760.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  zeerak["class"] = zeerak["class"].replace({"sexism": 1, "racism": 1, "none": 0})


class
0    7060
1    2588
Name: count, dtype: int64

In [16]:
# Rename the columns to match the other datasets
zeerak = zeerak.rename(columns={'text': 'tweet'})
zeerak.head()

,class,tweet
0,1,So Drasko just said he was impressed the girls...
1,1,Drasko they didn't cook half a bird you idiot ...
2,1,Hopefully someone cooks Drasko in the next ep ...
3,1,of course you were born in serbia...you're as ...
4,1,RT @YesYoureRacist: At least you're only a tin...


In [17]:
print("Rows:", zeerak.shape[0])
print("Columns:", zeerak.shape[1])

Rows: 9648
Columns: 2


### **Concatenation**

In [18]:
data = pd.concat([davidson, hasoc, zeerak], axis=0, ignore_index=True)
data.head()

,class,tweet
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [19]:
data["class"].value_counts()

class
1    25469
0    14814
Name: count, dtype: int64

In [20]:
print("Rows:", data.shape[0])
print("Columns:", data.shape[1])

Rows: 40283
Columns: 2


### **Cleaning**

In [21]:
#Check point
# Creation of one dataset for embedding of text into a vector and another one for embbeding with GloVe + Twitter
data_cleaning = data.copy()
data_cleaning_slight = data.copy()

In [22]:
# Transform emojis into words

def emoji_to_words(text):
  return emoji.demojize(text, language='en')

data_cleaning['tweet'] = data_cleaning['tweet'].apply(emoji_to_words)
data_cleaning_slight['tweet'] = data_cleaning_slight['tweet'].apply(emoji_to_words)

In [23]:
# Remove URLs from tweets

def remove_urls(text):
  return re.sub(r'http\S+', '', text)

data_cleaning['tweet'] = data_cleaning['tweet'].apply(remove_urls)
data_cleaning_slight['tweet'] = data_cleaning_slight['tweet'].apply(remove_urls)

In [24]:
# Remove mentions from tweets
def remove_mentions(text):
  return re.sub(r'@\w+', '', text)

data_cleaning['tweet'] = data_cleaning['tweet'].apply(remove_mentions)


In [25]:
# Remove symbols from tweets

def leave_letters(text):
  text = re.sub(r'[^a-zA-Z]', ' ', text)
  text = re.sub(r'\s+', ' ', text)
  return text

data_cleaning['tweet'] = data_cleaning['tweet'].apply(leave_letters)

In [26]:
# Remove uncommon symbols from tweets

def clean_tweet_for_glove(text):
  # Keep only letters, numbers, hashtags, mentions, apostrophes, emojis, and basic punctuation
  text = re.sub(r"[^a-zA-Z0-9@#'’!?.,\s]", '', text)  # keep @, #, apostrophes, basic punctuation
  text = re.sub(r'\s+', ' ', text)  # replace multiple spaces with single space
  return text.strip()

data_cleaning_slight['tweet'] = data_cleaning_slight['tweet'].apply(clean_tweet_for_glove)

In [27]:
# Remove symbols from tweets

def lowercase(text):
  return text.lower()

data_cleaning['tweet'] = data_cleaning['tweet'].apply(lowercase)
data_cleaning_slight['tweet'] = data_cleaning_slight['tweet'].apply(lowercase)

In [28]:
# Correct spelling
spell = SpellChecker()

def correct_spelling(text):
  words = text.split()
  corrected_words = [spell.correction(word) or word for word in words]
  return ' '.join(corrected_words)

data_cleaning['tweet'] = [correct_spelling(text) for text in tqdm(data_cleaning['tweet'])]

100%|███████████████████████████████████████████████████████████████████████████████████████| 40283/40283 [2:47:53<00:00,  4.00it/s]


In [29]:
# Remove stopwords

def remove_stopwords(text):
  stop_words = set(stopwords.words('english'))  # Use English stop words
  words = text.split()
  filtered_words = [word for word in words if word not in stop_words]
  return " ".join(filtered_words)

data_cleaning['tweet'] = data_cleaning['tweet'].apply(remove_stopwords)

In [30]:
# Stemming the words

stemmer = PorterStemmer()

def stem_text(text):
  words = text.split()
  stemmed_words = [stemmer.stem(word) for word in words]
  return " ".join(stemmed_words)

  data_cleaning['tweet'] = data_cleaning['tweet'].apply(stem_text)


In [31]:
data_cleaning['tweet'].head(200)

0      woman complain cleaning house amp man always t...
1             boy days cold toga bad coffin hoe st place
2           dawn ever fuck bitch start cry confused shit
3                                       look like granny
4         shit hear might true might faker bitch told ya
                             ...                        
195    tired bitches saying look mean night big af we...
196    birds grandads may never see thanks climate ch...
197                                 stay beautiful bitch
198    wutkinda r purple ceeeleee man girl playing st...
199          money getting taller bitches getting blurry
Name: tweet, Length: 200, dtype: object

In [32]:
data_cleaning_slight['tweet'].head(200)

0      !!! rt @mayasolovely as a woman you shouldn't ...
1      !!!!! rt @mleew17 boy dats cold...tyga dwn bad...
2      !!!!!!! rt @urkindofbrand dawg!!!! rt @80sbaby...
3      !!!!!!!!! rt @cganderson @vivabased she look l...
4      !!!!!!!!!!!!! rt @shenikaroberts the shit you ...
                             ...                        
195    @montrell i'm tired of bitches saying i look m...
196    @motherjones 10 birds your grandkids may never...
197                 @mvckfadden stay beautiful you bitch
198    @nickiminaj #wutkinda r purple. ceeeleeeman th...
199    @nastycopper money getting taller and bitches ...
Name: tweet, Length: 200, dtype: object

In [33]:
# Check and remove duplicates
def check_remove_dup(dataset):
  print(f'Duplicates: {dataset["tweet"].duplicated().sum()}')
  duplicated_tweets = dataset["tweet"].duplicated()
  return dataset[~duplicated_tweets]

In [34]:
# Check and remove duplicates in the first dataset

data_cleaning = check_remove_dup(data_cleaning)

Duplicates: 1823


In [35]:
# Check and remove duplicates in the second dataset

data_cleaning_slight = check_remove_dup(data_cleaning_slight)


Duplicates: 185


In [36]:
# Check for null or empty 

def check_null_empty(dataset):
  keep = ~((dataset["tweet"].isnull()) | (dataset["tweet"] == ""))
    
  print(f'Number of nulls or empty: {(~keep).sum()}')
    
  dataset = dataset[keep]
    
  print(f'Cleaned data shape: {dataset.shape}')

  return dataset

In [37]:
# Check for null or empty in the first dataset

data_cleaning = check_null_empty(data_cleaning)

Number of nulls or empty: 1
Cleaned data shape: (38459, 2)


In [38]:
# Check for null or empty in the second dataset

data_cleaning_slight = check_null_empty(data_cleaning_slight)

Number of nulls or empty: 1
Cleaned data shape: (40097, 2)


In [39]:
# Checking the dataset's balance

data_cleaning["class"].value_counts()

class
1    24345
0    14114
Name: count, dtype: int64

In [40]:
# Checking the dataset's balance

data_cleaning_slight["class"].value_counts()

class
1    25419
0    14678
Name: count, dtype: int64

In [41]:
# Function to get a random synonym of a word

def get_synonym(word):
  synonyms = []
  for syn in wordnet.synsets(word):
    for lemma in syn.lemmas():
      synonyms.append(lemma.name())
  if len(synonyms) > 0:
    synonyms = list(set(synonyms))
    return synonyms[random.randint(0, len(synonyms) - 1)]
  else:
    return ""

In [42]:
# Custom random insertion function

def random_synonym_insert_augment(text, alpha):

  words = text.split()
  new_text = words.copy()
  for word in words:
    if random.random() < alpha:
      synonym = get_synonym(word)
      if synonym != "":
        position = random.randint(0, len(new_text) - 1)
        new_text.insert(position, synonym)

  return [" ".join(new_text)]

In [43]:
# Balance the data set using easy data augmentation

#Choose an alpha parameter (Percentage of words in a sentence that are changed)
alpha = 0.25

aug_synonym = naw.SynonymAug(aug_src='wordnet', aug_p = alpha)
aug_swap = naw.RandomWordAug(action="swap", aug_p = alpha)
aug_delete = naw.RandomWordAug(action="delete", aug_p = alpha)


def data_augmentation(dataset, alpha):
  # Store the new rows
  new_rows = []

  # Size to balance the classes
  desired_size = len(dataset[dataset['class'] == 1]) - len(dataset[dataset['class'] == 0])

  non_toxic = dataset[dataset['class'] == 0]

  # Until balanced
  while len(new_rows) < desired_size:
    # Get a random sample from the minority class
    random_row = non_toxic.sample(1)

    # Pick a random EDA technique and apply it
    random_num = random.randint(1, 4)
    augmented_text = []

    if random_num == 1:
      augmented_text = aug_synonym.augment(random_row['tweet'].values[0])
    elif random_num == 2:
      augmented_text = aug_swap.augment(random_row['tweet'].values[0])
    elif random_num == 3:
      augmented_text = random_synonym_insert_augment(random_row['tweet'].values[0], alpha)
    else:
      augmented_text = aug_delete.augment(random_row['tweet'].values[0])

    if len(augmented_text) > 0:
      new_rows.append({ "tweet": augmented_text[0], "class": random_row['class'].values[0] })


  # New rows dataframe
  new_rows_df = pd.DataFrame(new_rows)

  # Concatenate the datasets
  return pd.concat([dataset, new_rows_df], ignore_index = True, axis = 0)

In [44]:
# Balance the first dataset

balanced_data = data_augmentation(data_cleaning, alpha)

print(f'Balanced data shape: {balanced_data.shape}')
print()
balanced_data["class"].value_counts()


Balanced data shape: (48690, 2)



class
0    24345
1    24345
Name: count, dtype: int64

In [45]:
# Balance the second dataset

balanced_data_glove = data_augmentation(data_cleaning_slight, alpha)

print(f'Balanced data shape: {balanced_data_glove.shape}')
print()
balanced_data_glove["class"].value_counts()

Balanced data shape: (50838, 2)



class
0    25419
1    25419
Name: count, dtype: int64

In [46]:
# Check and remove duplicates again that could have appeared due to augmentation

balanced_data = check_remove_dup(balanced_data)

Duplicates: 994


In [47]:
# Check and remove duplicates again that could have appeared due to augmentation in the second dataset

balanced_data_glove = check_remove_dup(balanced_data_glove)

Duplicates: 655


In [48]:
# Check for null or empty again if they appeared due to augmentation

balanced_data = check_null_empty(balanced_data)

Number of nulls or empty: 0
Cleaned data shape: (47696, 2)


In [49]:
# Check for null or empty again if they appeared due to augmentation in the second dataset

balanced_data_glove = check_null_empty(balanced_data_glove)

Number of nulls or empty: 0
Cleaned data shape: (50183, 2)


In [50]:
# Final distribution

balanced_data["class"].value_counts()

class
1    24345
0    23351
Name: count, dtype: int64

In [51]:
# Final distribution

balanced_data_glove["class"].value_counts()

class
1    25419
0    24764
Name: count, dtype: int64

In [52]:
balanced_data.to_csv('balanced_data.csv', index=False)

In [53]:
balanced_data_glove.to_csv('balanced_data_glove.csv', index=False)

## **Embeddings Secuenciales roBERTa Twitter**

In [5]:
# Setting the device to use GPU

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In [6]:
# Instanciating pre trained model

model_name = "cardiffnlp/twitter-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()
model.to(device)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [12]:
# Reading the stored data

data = pd.read_csv('balanced_data.csv')
tweets = data['tweet'].astype(str).values

In [22]:
# Generating the embeddings

embeddings = []

for tweet in tqdm(tweets, desc = "Generando embeddings..."):
    inputs = tokenizer(tweet, padding='max_length', max_length=32, truncation = True, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    embeddings.append(outputs.last_hidden_state.squeeze(0))

embeddings_tensor = torch.stack(embeddings).cpu().numpy()
    


Generando embeddings...: 100%|███████████| 47696/47696 [01:41<00:00, 467.69it/s]


In [23]:
# Storing the embeddings

np.save('embeddings_data_sequencial.npy', embeddings_tensor)

## **Embeddings Contextuales BERT**

In [76]:
model_name = 'cardiffnlp/twitter-roberta-base'
max_len = 64

tokenizer_roberta = AutoTokenizer.from_pretrained(model_name)
model_roberta = AutoModel.from_pretrained(model_name)
model_roberta.eval()

NameError: name 'init_empty_weights' is not defined

In [ ]:
data = pd.read_csv('data/balanced_data.csv')
tweets = data['tweet'].astype(str).tolist()

In [ ]:
def mean_pooling(outputs, attention_mask):
    token_embeddings = outputs.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
    sum_mask = input_mask_expanded.sum(1)
    return sum_embeddings / sum_mask

In [ ]:
def generate_embeddings(texts, model, tokenizer, max_len=64, device=None):
    
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    embeddings_cls = []
    embeddings_mean = []

    for text in tqdm(texts, desc='Generando embeddings'):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=max_len)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)

        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
        embeddings_cls.append(cls_embedding)

        mean_embedding = mean_pooling(outputs, inputs['attention_mask']).squeeze().cpu().numpy()
        embeddings_mean.append(mean_embedding)

    return embeddings_cls, embeddings_mean

In [ ]:
embeddings_cls, embeddings_mean = generate_embeddings(tweets, model_roberta, tokenizer_roberta, max_len)

data["embedding_cls"] = [vec.tolist() for vec in embeddings_cls]
data["embedding_mean"] = [vec.tolist() for vec in embeddings_mean]

data.to_csv('data/embeddings_data.csv',index=False)

In [ ]:
df = pd.read_csv("data/embeddings_data.csv")
df["embedding_mean"] = df["embedding_mean"].apply(ast.literal_eval)
X = np.vstack(df["embedding_mean"].values)
labels = df["class"].astype(str) if "class" in df.columns else None

reducer = umap.UMAP(n_components=3, random_state=42)
X_umap = reducer.fit_transform(X)

fig = px.scatter_3d(
    x=X_umap[:, 0],
    y=X_umap[:, 1],
    z=X_umap[:, 2],
    color=labels,
    title="UMAP 3D de embeddings de tweets",
    labels={"color": "Clase"}
)
fig.show()